# Установка требуемых пакетов

In [1]:
#!g1.1
%pip install torch
%pip install pytorch-lightning==0.9.0
%pip install transformers
%pip install tokenizers==0.10.2

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 2.6 MB 2.5 MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [2]:
#!g1.1
import pytorch_lightning as pl
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from tokenizers import BertWordPieceTokenizer
from torch import Tensor
from torch.nn.modules import CrossEntropyLoss, BCEWithLogitsLoss
from torch.utils.data import DataLoader
from transformers import AdamW
from torch.optim import SGD
from typing import Dict
import os
import tqdm
from tqdm import tqdm

In [3]:
#!g1.1
from datasets.mrc_ner_dataset import MRCNERDataset
from datasets.truncate_dataset import TruncateDataset
from datasets.mrc_ner_dataset import collate_to_max_length
from metrics.query_span_f1 import QuerySpanF1
from models.bert_query_ner import BertQueryNER
from models.query_ner_config import BertQueryNerConfig
from loss import *
from utils.get_parser import get_parser
from utils.random_seed import set_random_seed

In [4]:
#!g1.1
set_random_seed(0) # Для повторения исследований

# Задание параметров обучения и модели

In [5]:
#!g1.1
# Все параметры трейнера задаются в этой ячейке

trainer_args = {
    "default_root_dir" : "logs" , # Куда сохранять модели, логи и т.д.
    "max_epochs" : 16 , # Число эпох для обучения
    "resume_from_checkpoint" : None , # Воспроизвести обучение с чекпойнта
    "val_check_interval" : 0.5 , # Как часто валидировать модель
    "gpus" : 1 # , # число используемых видеокарт
    # Добавьте любые нужные аргументы для конфигурации модели. Их можно найти на 
    # https://pytorch-lightning.readthedocs.io/en/latest/common/trainer.html#trainer-flags
}


In [6]:
#!g1.1
# Часто изменяемые параметры (не меняющие саму модель) - в этой
CKPT_PATH = "saved_logs/model_10_08-20_47.ckpt" # Путь к чекпойнту, если используется
BERT_PATH = "for_mrc/rubert" # Путь к модели берта
DATA_PATH = "jsons" # Датасет

In [7]:
#!g1.1
model_args = {
    "data_dir" : DATA_PATH ,
    "bert_config_dir" : BERT_PATH ,
    "pretrained_checkpoint" : CKPT_PATH ,
    "max_length" : 128 , 
    "batch_size" : 32 , 
    "lr" : 2e-5 , 
    "workers" : 8 , 
    "weight_decay" : 0.01 , 
    "warmup_steps" : 0 , 
    "adam_epsilon" : 1e-8 , # Epsilon для алгоритма ADAMW
    "mrc_dropout" : 0.1 , # Dropout вероятность в модели MRC
    "weight_start" : 1.0 , # Коэффициент для стартовых позиций меток (альфа)
    "weight_end" : 1.0 , # Коэффициент для конечных позиций меток (бета)
    "weight_span" : 1.0 , # Коэффициент для спанов меток (гамма)
    "loss_type" : "bce" , 
    "optimizer" : "adamw" ,
    "dice_smooth" : 1e-8 ,
    "final_div_factor" : 1e4 ,
    "span_loss_candidates" : "all" , 
    "accumulate_grad_batches" : 1
}

In [8]:
#!g1.1
bert_args = {
    "bert_dropout" : 0.1 # Dropout самого берта
}

# Загрузка модели и данных

In [9]:
#!g1.1
from models.bert_labeling import BertLabeling

In [10]:
#!g1.1
# Проверка данных на корректность

dataset_path = os.path.join(DATA_PATH, f"train.json")
vocab_path = os.path.join(BERT_PATH, "vocab.txt") # важно знать, по какому словарю токенизировать
dataset = MRCNERDataset(dataset_path=dataset_path, 
                        tokenizer=BertWordPieceTokenizer(vocab_path, lowercase = False),
                        max_length=128,
                        pad_to_maxlen=False,
                        tag = None # для тестирования по конкретным классам сущностей
                        )

dataloader = DataLoader(
    dataset=dataset,
    batch_size=32,
    num_workers=8,
    shuffle=True,
    collate_fn=collate_to_max_length
)

for batch_idx in tqdm(dataloader):
    pass # Позволяет найти все опечатки, исправить их (ибо вызывается ошибка некорректной токенизации в противном случае)
    # В данный момент все такие опечатки обнуляются

print("All correct!")

100%|██████████| 8968/8968 [00:55<00:00, 162.69it/s]


All correct!


In [11]:
#!g1.1
torch.cuda.empty_cache()

In [12]:
#!g1.1
model = BertLabeling(model_args, bert_args, trainer_args) # Инициализиуем модель на их основе

# Если грузим из чекпойнта
if model_args["pretrained_checkpoint"]:
    model.load_state_dict(torch.load(model_args["pretrained_checkpoint"], 
                                     map_location=torch.device('cpu'))["state_dict"]) 

Some weights of BertQueryNER were not initialized from the model checkpoint at for_mrc/rubert and are newly initialized: ['span_embedding.classifier2.bias', 'end_outputs.bias', 'start_outputs.weight', 'start_outputs.bias', 'end_outputs.weight', 'span_embedding.classifier2.weight', 'span_embedding.classifier1.weight', 'span_embedding.classifier1.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: Error(s) in loading state_dict for BertLabeling:
	Missing key(s) in state_dict: "model.bert.embeddings.position_ids". 

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:828: UserWarning: The following variables cannot be serialized: model
  warnings.warn(message)


# Обучение


In [14]:
#!g1.1
checkpoint_callback = ModelCheckpoint(
    # Директория, куда будут сохраняться чекпойнты и логи (по умолчанию корневая папка проекта)
    filepath=trainer_args["default_root_dir"], 
    save_top_k=5, # Сохранять топ 5 моделей по метрике monitor
    verbose=True, # Уведомлять о результатах валидации
    monitor="span_f1", # Метрика для подсчета качества модели, см. span_f1
    period=-1, # Сохранять чекпойнты каждую эпоху
    mode="max", # Сохраняем самые максимальные по метрике модели
)

# Инициализация Trainer на основе аргументов командной строки 
# Настройка сохранения моделей через callbacks
trainer = Trainer(**trainer_args, checkpoint_callback = checkpoint_callback)

/home/jupyter/.local/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Checkpoint directory logs exists and is not empty with save_top_k != 0.All files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [15]:
#!g1.1
trainer.fit(model) # Запуск процесса обучения и валидации, с мониторингом

1

{'id': '7395.22', 'context': 'Неизвестный поклонник Эдгара ПоПерезахоронение Эдгара По.', 'tag': 'PERSON', 'query': 'Человек - мужчина, женщина или ребенок.', 'filename': '3948_text', 'exists': True, 'start_positions': [22, 47], 'end_positions': [31, 56], 'span_positions': ['22;31', '47;56'], 'spans': ['Эдгара По', 'Эдгара По']}
{'id': '8997.6', 'context': 'Кроме того, сообщалось, что 69-летний Т.Чхеидзе, который руководил БДТ с 2007г., уже выставил на продажу свою петербургскую квартиру и собирается вернуться в Тбилиси, где живет его семья.', 'tag': 'DATE', 'query': 'Дата - это номер дня в месяце, часто указываемый в сочетании с названием дня, месяца и года.', 'filename': '633', 'exists': True, 'start_positions': [71], 'end_positions': [77], 'span_positions': ['71;77'], 'spans': ['с 2007']}
{'id': '5013.18', 'context': 'Самолёт «Як-18Т»\nСообщение об этой авиационной катастрофе поступило в ЦУКС МЧС России по Архангельской области в 16 часов 25 минут.', 'tag': 'NUMBER', 'query': 'Число

1

/home/jupyter/.local/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)

  | Name     | Type              | Params
-----------------------------------------------
0 | model    | BertQueryNER      | 180 M 
1 | bce_loss | BCEWithLogitsLoss | 0     
2 | span_f1  | QuerySpanF1       | 0     
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:155: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)
/home/jupyter/.local/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Detected KeyboardInterrupt, atte

# Тестирование

In [ ]:
#!g1.1
# Задать пути к модели, данным и словарю
CHECKPOINTS = "saved_logs/model_10_08-20_47.ckpt"
DATASET_PATH = "jsons/test.json"
VOCAB_PATH = "for_mrc/rubert/vocab.txt"

In [ ]:
#!g1.1
checkpoint_callback = ModelCheckpoint(
    filepath=CHECKPOINTS, 
    save_top_k=-1, # не сохранять
    verbose=False, # не уведомлять
    monitor="span_f1", 
    period=1,
    mode="max",
)

trainer = Trainer(**trainer_args, checkpoint_callback = checkpoint_callback)

In [ ]:
#!g1.1
model.output_test_file = open("test_dataset.out", "w", encoding = "utf-8")
trainer.test(model, model.get_dataloader("test", tag = None)) # Тестирование по всем классам

In [15]:
#!g1.1
model.output_test_file = open("test_dataset.out", "w", encoding = "utf-8")
trainer.test(model, model.get_dataloader("test", tag = "PERSON")) # Тестирование по отдельным классам

/home/jupyter/.local/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  warnings.warn(*args, **kwargs)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'span_f1': tensor(0.9674, device='cuda:0'),
 'span_fn': tensor(27, device='cuda:0'),
 'span_fp': tensor(36, device='cuda:0'),
 'span_precision': tensor(0.9629, device='cuda:0'),
 'span_recall': tensor(0.9719, device='cuda:0'),
 'span_tp': tensor(935, device='cuda:0'),
 'val_loss': tensor(0.0106, device='cuda:0')}
--------------------------------------------------------------------------------



[{'val_loss': 0.010555455461144447,
  'span_precision': 0.9629248380661011,
  'span_recall': 0.9719334840774536,
  'span_f1': 0.9674081802368164,
  'span_tp': 935,
  'span_fp': 36,
  'span_fn': 27}]

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:828: UserWarning: The following variables cannot be serialized: checkpoint_callback, model, trainer
  warnings.warn(message)


In [ ]:
#!g1.1
